In [10]:
import pandas as pd

# Đọc dữ liệu 14 năm từ file Excel
df = pd.read_excel("data_nckh.xlsx", parse_dates=["Time"])
df = df.sort_values("Time")  # sắp xếp theo thời gian

# Kiểm tra và xử lý dữ liệu missing/duplicate
assert df["Time"].is_unique, "Thời gian bị trùng lặp!"  # đảm bảo không trùng
assert df.isnull().sum().sum() == 0, "Có giá trị null trong dữ liệu!"  # đảm bảo không có null

# Lấy cột giá vàng Việt Nam làm biến mục tiêu, các cột khác làm đặc trưng
target_col = "Gold_Price_VN"
feature_cols = ["CPI_VN", "Gold_Price_World", "Oil_Price", "Tỷ giá USD/VND", "VN-Index", 
                "CPI_USA", "SM_M2", "SX_CN", "INTEREST_RATE", "GSI", "GSI_VN"]

# Chuẩn bị dữ liệu cho mô hình
features = df[feature_cols].values
target = df[target_col].values

print("Tổng số mẫu:", len(df))


Tổng số mẫu: 5479


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, shuffle=False, 
                                                    test_size=0.2)
print("Số mẫu train:", len(X_train), " | Số mẫu test:", len(X_test))


Số mẫu train: 4383  | Số mẫu test: 1096


In [12]:
import numpy as np

time_step = 30  # độ dài chuỗi đầu vào cho LSTM (30 ngày)

# Hàm tạo dữ liệu dạng sequence cho LSTM
def create_sequences(X, y, time_step=30):
    X_seq, y_seq = [], []
    for i in range(time_step, len(X)):
        X_seq.append(X[i-time_step:i])    # đoạn [i-time_step ... i-1]
        y_seq.append(y[i])                # giá trị tại thời điểm i (dự báo ở bước kế)
    return np.array(X_seq), np.array(y_seq)

# Tạo dữ liệu cho LSTM (sử dụng tập train)
X_train_seq, y_train_seq = create_sequences(X_train, y_train, time_step)
X_test_seq, y_test_seq = create_sequences(np.vstack([X_train[-time_step:], X_test]), 
                                         np.concatenate([y_train[-time_step:], y_test]), 
                                         time_step)
# Lưu ý: để tạo X_test_seq chính xác, nối đuôi 30 ngày cuối của train với test để không bị thiếu chuỗi
print("Shape X_train_seq:", X_train_seq.shape)  # (n_train_seq, time_step, n_features)


Shape X_train_seq: (4353, 30, 11)


In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Xây dựng model LSTM đơn giản
lstm_model = models.Sequential([
    layers.LSTM(50, activation='tanh', return_sequences=False, input_shape=(time_step, X_train_seq.shape[2])),
    layers.Dense(1)  # đầu ra một giá trị (giá vàng dự báo)
])
lstm_model.compile(optimizer='adam', loss='mse')
# Huấn luyện LSTM trên dữ liệu chuỗi
history = lstm_model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=32, 
                         validation_split=0.1, verbose=0)


c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
# Thêm đặc trưng lag-1 của giá vàng (giá vàng ngày trước đó)
X_rf_train = np.hstack([X_train[1:], y_train[:-1, None]])  # ghép cột giá vàng ngày trước vào features ngày hiện tại
y_rf_train = y_train[1:]
X_rf_test = np.hstack([X_test[1:], y_test[:-1, None]])
y_rf_test = y_test[1:]

from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
rf_model.fit(X_rf_train, y_rf_train)


RandomForestRegressor(random_state=0)

In [15]:
# Dự báo bằng LSTM trên tập test (sử dụng X_test_seq đã chuẩn bị)
y_pred_lstm = lstm_model.predict(X_test_seq).flatten()

# Dự báo bằng RF trên tập test
y_pred_rf = rf_model.predict(X_rf_test)


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


In [16]:
# Kết hợp dự báo (weighted average)
w_lstm, w_rf = 0.5203728882105332, 0.4539474722842706
#w_lstm, w_rf = 0.6421489507388553, 0.3872990332727839
y_pred_ensemble = w_lstm * y_pred_lstm[-len(y_rf_test):] + w_rf * y_pred_rf  # đảm bảo độ dài tương ứng

# Nếu độ dài y_pred_lstm_seq > y_pred_rf (do cách tạo chuỗi), cắt cho khớp với RF (bỏ vài điểm đầu)


In [17]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

r2 = r2_score(y_rf_test, y_pred_ensemble)
rmse = np.sqrt(mean_squared_error(y_rf_test, y_pred_ensemble))
mae = mean_absolute_error(y_rf_test, y_pred_ensemble)
print(f"Ensemble model – R^2: {r2:.3f}, RMSE: {rmse:.3f}, MAE: {mae:.3f}")


Ensemble model – R^2: 0.689, RMSE: 8.558, MAE: 6.597


In [18]:
print("LSTM – R^2: %.3f, RMSE: %.3f, MAE: %.3f" % (
    r2_score(y_test_seq, y_pred_lstm), 
    np.sqrt(mean_squared_error(y_test_seq, y_pred_lstm)), 
    mean_absolute_error(y_test_seq, y_pred_lstm)
))
print("Random Forest – R^2: %.3f, RMSE: %.3f, MAE: %.3f" % (
    r2_score(y_rf_test, y_pred_rf), 
    np.sqrt(mean_squared_error(y_rf_test, y_pred_rf)), 
    mean_absolute_error(y_rf_test, y_pred_rf)
))


LSTM – R^2: -0.001, RMSE: 15.355, MAE: 12.582
Random Forest – R^2: 0.999, RMSE: 0.569, MAE: 0.291


In [31]:
df = pd.read_excel("data_nckh.xlsx")
print(df.columns)

Index(['Time', 'Gold_Price_VN', 'CPI_VN', 'Gold_Price_World', 'Oil_Price',
       'Tỷ giá USD/VND', 'SX_CN', 'VN-Index', 'CPI_USA', 'SM_M2',
       'INTEREST_RATE', 'GSI', 'GSI_VN'],
      dtype='object')


# Sử dụng Bayesian Optimization

In [ ]:
LSTM - units: 165, learning_rate: 0.0027600131887437498
RF - n_estimators: 300, max_depth: 50, min_samples_split: 2 

In [51]:
import numpy as np
import pandas as pd
from hyperopt import fmin, tpe, hp, Trials
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Đọc dữ liệu và tiền xử lý
df = pd.read_excel("data_nckh.xlsx", parse_dates=["Time"])
df = df.sort_values("Time")
df = df.drop_duplicates(subset=["Time"])  # Đảm bảo không có giá trị duplicate

# Chọn các cột đặc trưng và mục tiêu
target_col = "Gold_Price_VN"
feature_cols = ["CPI_VN", "Gold_Price_World", "Oil_Price", "Tỷ giá USD/VND", "VN-Index", 
                "CPI_USA", "SM_M2", "SX_CN", "INTEREST_RATE", "GSI", "GSI_VN"]

features = df[feature_cols].values
target = df[target_col].values

# Tách dữ liệu train và test
X_train, X_test, y_train, y_test = train_test_split(features, target, shuffle=False, test_size=0.2)

# Chuẩn bị dữ liệu cho LSTM
time_step = 30  # Sử dụng 30 ngày làm bước thời gian cho LSTM

def create_sequences(X, y, time_step=30):
    X_seq, y_seq = [], []
    for i in range(time_step, len(X)):
        X_seq.append(X[i-time_step:i])  # X[i-time_step ... i-1]
        y_seq.append(y[i])             # Dự báo tại thời điểm i
    return np.array(X_seq), np.array(y_seq)

X_train_seq, y_train_seq = create_sequences(X_train, y_train, time_step)
X_test_seq, y_test_seq = create_sequences(np.vstack([X_train[-time_step:], X_test]), 
                                         np.concatenate([y_train[-time_step:], y_test]), 
                                         time_step)

# Định nghĩa mô hình LSTM
def create_lstm_model(units, lr):
    model = Sequential()
    model.add(LSTM(units=units, activation='tanh', return_sequences=False, input_shape=(time_step, X_train_seq.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse')
    return model

# Hàm mục tiêu cho Bayesian Optimization
def objective(params):
    # Huấn luyện mô hình LSTM
    lstm_model = create_lstm_model(units=165, lr=0.0027600131887437498)
    history = lstm_model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=32, validation_split=0.1, verbose=0)
    
    # Thêm đặc trưng lag-1 của giá vàng (giá vàng ngày trước đó)
    X_rf_train = np.hstack([X_train[1:], y_train[:-1, None]])  # ghép cột giá vàng ngày trước vào features ngày hiện tại
    y_rf_train = y_train[1:]
    X_rf_test = np.hstack([X_test[1:], y_test[:-1, None]])
    y_rf_test = y_test[1:]
    
    # Dự báo bằng LSTM trên tập test
    y_pred_lstm = lstm_model.predict(X_test_seq).flatten()
    
    # Huấn luyện Random Forest
    rf_model = RandomForestRegressor(n_estimators=300, max_depth=50, random_state=0,min_samples_split=2)
    rf_model.fit(X_rf_train, y_rf_train)
    
    # Dự báo bằng Random Forest trên tập test
    y_pred_rf = rf_model.predict(X_rf_test)
    
    # Kết hợp dự báo từ LSTM và RF (weighted average)
    y_pred_ensemble = params['w_lstm'] * y_pred_lstm[-len(y_rf_test):] + params['w_rf'] * y_pred_rf
    
    # Đánh giá mô hình bằng các chỉ số
    r2 = r2_score(y_rf_test, y_pred_ensemble)
    rmse = np.sqrt(mean_squared_error(y_rf_test, y_pred_ensemble))
    mae = mean_absolute_error(y_rf_test, y_pred_ensemble)
    
    return rmse  # Ta tối ưu RMSE để cải thiện độ chính xác

# Không gian tham số cho LSTM và Random Forest
space = {
    'w_lstm': hp.uniform('w_lstm', 0.5, 1.0),  # Trọng số cho LSTM
    'w_rf': hp.uniform('w_rf', 0.0, 0.5)      # Trọng số cho Random Forest
}


# Tinh chỉnh mô hình bằng Bayesian Optimization
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20, trials=trials)

print("Best hyperparameters:", best)

# Huấn luyện lại mô hình với tham số tối ưu
lstm_model = create_lstm_model(units=165, lr=0.0027600131887437498)
lstm_model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=32, validation_split=0.1, verbose=0)

# Dự báo với mô hình LSTM
y_pred_lstm = lstm_model.predict(X_test_seq).flatten()

# Huấn luyện lại Random Forest với tham số tối ưu
rf_model = RandomForestRegressor(n_estimators=300, max_depth=50, random_state=0, min_samples_split=2)
rf_model.fit(X_train, y_train)

# Dự báo với Random Forest
y_pred_rf = rf_model.predict(X_test)

# Kết hợp dự báo từ LSTM và RF (weighted average)
y_pred_ensemble = best['w_lstm'] * y_pred_lstm + best['w_rf'] * y_pred_rf

# Đánh giá kết quả cuối cùng
r2 = r2_score(y_rf_test, y_pred_ensemble)
rmse = np.sqrt(mean_squared_error(y_rf_test, y_pred_ensemble))
mae = mean_absolute_error(y_rf_test, y_pred_ensemble)

print(f"Final Ensemble model – R^2: {r2:.3f}, RMSE: {rmse:.3f}, MAE: {mae:.3f}")


  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



 1/35 ━━━━━━━━━━━━━━━━━━━━ 6s 194ms/step
 9/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step  
16/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
23/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
31/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step

  5%|▌         | 1/20 [01:33<29:40, 93.71s/trial, best loss: 9.323040331533665]

c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



 1/35 ━━━━━━━━━━━━━━━━━━━━ 5s 175ms/step              
11/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step      
21/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step      
31/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step      
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step     
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step      

 10%|█         | 2/20 [03:05<27:42, 92.34s/trial, best loss: 9.323040331533665]

...

c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



 1/35 ━━━━━━━━━━━━━━━━━━━━ 7s 210ms/step                
 7/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step        
10/35 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step       
14/35 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step        
21/35 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step        
29/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step         
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step       
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step        

100%|██████████| 20/20 [38:03<00:00, 114.16s/trial, best loss: 8.619275348120329]
Best hyperparameters: {'w_lstm': 0.5952718585197654, 'w_rf': 0.4585406693109709}


c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


ValueError: Found input variables with inconsistent numbers of samples: [1095, 1096]

In [50]:
import pandas as pd

# Đọc dữ liệu 14 năm từ file Excel
df = pd.read_excel("data_nckh.xlsx", parse_dates=["Time"])
df = df.sort_values("Time")  # sắp xếp theo thời gian

# Kiểm tra và xử lý dữ liệu missing/duplicate
assert df["Time"].is_unique, "Thời gian bị trùng lặp!"  # đảm bảo không trùng
assert df.isnull().sum().sum() == 0, "Có giá trị null trong dữ liệu!"  # đảm bảo không có null

# Lấy cột giá vàng Việt Nam làm biến mục tiêu, các cột khác làm đặc trưng
target_col = "Gold_Price_VN"
feature_cols = ["CPI_VN", "Gold_Price_World", "Oil_Price", "Tỷ giá USD/VND", "VN-Index", 
                "CPI_USA", "SM_M2", "SX_CN", "INTEREST_RATE", "GSI", "GSI_VN"]

# Chuẩn bị dữ liệu cho mô hình
features = df[feature_cols].values
target = df[target_col].values

print("Tổng số mẫu:", len(df))

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, shuffle=False, 
                                                    test_size=0.2)
print("Số mẫu train:", len(X_train), " | Số mẫu test:", len(X_test))

import numpy as np

time_step = 30  # độ dài chuỗi đầu vào cho LSTM (30 ngày)

# Hàm tạo dữ liệu dạng sequence cho LSTM
def create_sequences(X, y, time_step=30):
    X_seq, y_seq = [], []
    for i in range(time_step, len(X)):
        X_seq.append(X[i-time_step:i])    # đoạn [i-time_step ... i-1]
        y_seq.append(y[i])                # giá trị tại thời điểm i (dự báo ở bước kế)
    return np.array(X_seq), np.array(y_seq)

# Tạo dữ liệu cho LSTM (sử dụng tập train)
X_train_seq, y_train_seq = create_sequences(X_train, y_train, time_step)
X_test_seq, y_test_seq = create_sequences(np.vstack([X_train[-time_step:], X_test]), 
                                         np.concatenate([y_train[-time_step:], y_test]), 
                                         time_step)
# Lưu ý: để tạo X_test_seq chính xác, nối đuôi 30 ngày cuối của train với test để không bị thiếu chuỗi
print("Shape X_train_seq:", X_train_seq.shape)  # (n_train_seq, time_step, n_features)

import tensorflow as tf
from tensorflow.keras import layers, models

# Xây dựng model LSTM đơn giản
def create_lstm_model(units, lr):
    model = Sequential()
    model.add(LSTM(units=units, activation='tanh', return_sequences=False, input_shape=(time_step, X_train_seq.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse')
    return model

# Huấn luyện mô hình LSTM
lstm_model = create_lstm_model(units=165, lr=0.0027600131887437498)
    
# Dự báo bằng LSTM trên tập test
y_pred_lstm = lstm_model.predict(X_test_seq).flatten()

# Huấn luyện LSTM trên dữ liệu chuỗi
history = lstm_model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=32, 
                         validation_split=0.1, verbose=0)
# Thêm đặc trưng lag-1 của giá vàng (giá vàng ngày trước đó)
X_rf_train = np.hstack([X_train[1:], y_train[:-1, None]])  # ghép cột giá vàng ngày trước vào features ngày hiện tại
y_rf_train = y_train[1:]
X_rf_test = np.hstack([X_test[1:], y_test[:-1, None]])
y_rf_test = y_test[1:]

from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=300, max_depth=50, random_state=0,min_samples_split=2 )
rf_model.fit(X_rf_train, y_rf_train)

# Dự báo bằng LSTM trên tập test (sử dụng X_test_seq đã chuẩn bị)
y_pred_lstm = lstm_model.predict(X_test_seq).flatten()

# Dự báo bằng RF trên tập test
y_pred_rf = rf_model.predict(X_rf_test)

# Kết hợp dự báo (weighted average)
w_lstm, w_rf = 0.5203728882105332, 0.4539474722842706
y_pred_ensemble = w_lstm * y_pred_lstm[-len(y_rf_test):] + w_rf * y_pred_rf  # đảm bảo độ dài tương ứng
# Nếu độ dài y_pred_lstm_seq > y_pred_rf (do cách tạo chuỗi), cắt cho khớp với RF (bỏ vài điểm đầu)

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

r2 = r2_score(y_rf_test, y_pred_ensemble)
rmse = np.sqrt(mean_squared_error(y_rf_test, y_pred_ensemble))
mae = mean_absolute_error(y_rf_test, y_pred_ensemble)
print(f"Ensemble model – R^2: {r2:.3f}, RMSE: {rmse:.3f}, MAE: {mae:.3f}")

print("LSTM – R^2: %.3f, RMSE: %.3f, MAE: %.3f" % (
    r2_score(y_test_seq, y_pred_lstm), 
    np.sqrt(mean_squared_error(y_test_seq, y_pred_lstm)), 
    mean_absolute_error(y_test_seq, y_pred_lstm)
))
print("Random Forest – R^2: %.3f, RMSE: %.3f, MAE: %.3f" % (
    r2_score(y_rf_test, y_pred_rf), 
    np.sqrt(mean_squared_error(y_rf_test, y_pred_rf)), 
    mean_absolute_error(y_rf_test, y_pred_rf)
))

Tổng số mẫu: 5479
Số mẫu train: 4383  | Số mẫu test: 1096
Shape X_train_seq: (4353, 30, 11)


c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Ensemble model – R^2: 0.689, RMSE: 8.563, MAE: 6.596
LSTM – R^2: -0.001, RMSE: 15.356, MAE: 12.574
Random Forest – R^2: 0.999, RMSE: 0.565, MAE: 0.288


# Stacking

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.ensemble import RandomForestRegressor

# Đọc dữ liệu 14 năm từ file Excel
df = pd.read_excel("data_nckh.xlsx", parse_dates=["Time"])
df = df.sort_values("Time")  # sắp xếp theo thời gian

# Kiểm tra và xử lý dữ liệu missing/duplicate
assert df["Time"].is_unique, "Thời gian bị trùng lặp!"  # đảm bảo không trùng
assert df.isnull().sum().sum() == 0, "Có giá trị null trong dữ liệu!"  # đảm bảo không có null

# Lấy cột giá vàng Việt Nam làm biến mục tiêu, các cột khác làm đặc trưng
target_col = "Gold_Price_VN"
feature_cols = ["CPI_VN", "Gold_Price_World", "Oil_Price", "Tỷ giá USD/VND", "VN-Index", 
                "CPI_USA", "SM_M2", "SX_CN", "INTEREST_RATE", "GSI", "GSI_VN"]

# Chuẩn bị dữ liệu cho mô hình
features = df[feature_cols].values
target = df[target_col].values

print("Tổng số mẫu:", len(df))

# Tạo dữ liệu train và test
X_train, X_test, y_train, y_test = train_test_split(features, target, shuffle=False, test_size=0.2)
print("Số mẫu train:", len(X_train), " | Số mẫu test:", len(X_test))

# Tạo dữ liệu chuỗi cho LSTM
time_step = 30  # độ dài chuỗi đầu vào cho LSTM

def create_sequences(X, y, time_step=30):
    X_seq, y_seq = [], []
    for i in range(time_step, len(X)):
        X_seq.append(X[i-time_step:i])    # đoạn [i-time_step ... i-1]
        y_seq.append(y[i])                # giá trị tại thời điểm i (dự báo ở bước kế)
    return np.array(X_seq), np.array(y_seq)

X_train_seq, y_train_seq = create_sequences(X_train, y_train, time_step)
X_test_seq, y_test_seq = create_sequences(np.vstack([X_train[-time_step:], X_test]), 
                                         np.concatenate([y_train[-time_step:], y_test]), 
                                         time_step)

# Xây dựng mô hình LSTM
lstm_model = models.Sequential([
    layers.LSTM(50, activation='tanh', return_sequences=False, input_shape=(time_step, X_train_seq.shape[2])),
    layers.Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mse')

# Huấn luyện LSTM
lstm_model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=32, validation_split=0.1, verbose=0)

# Dự báo bằng LSTM trên tập test
y_pred_lstm = lstm_model.predict(X_test_seq).flatten()

# Tạo đặc trưng cho RandomForest
X_rf_train = np.hstack([X_train[1:], y_train[:-1, None]])  # ghép cột giá vàng ngày trước vào features ngày hiện tại
y_rf_train = y_train[1:]
X_rf_test = np.hstack([X_test[1:], y_test[:-1, None]])
y_rf_test = y_test[1:]

# Xây dựng mô hình RandomForest
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
rf_model.fit(X_rf_train, y_rf_train)

# Dự báo bằng RandomForest trên tập test
y_pred_rf = rf_model.predict(X_rf_test)

# Cắt bớt phần dư thừa của y_pred_lstm để khớp với y_rf_test
y_pred_lstm = y_pred_lstm[-len(y_rf_test):]

# Đảm bảo rằng y_pred_rf và y_pred_lstm có cùng chiều dài
y_pred_rf = y_pred_rf[-len(y_pred_lstm):]

# Đảm bảo rằng y_test_seq và y_pred_lstm có cùng chiều dài
y_test_seq = y_test_seq[-len(y_pred_lstm):]

# Xây dựng mô hình meta (Linear Regression)
X_meta_train = np.vstack([y_pred_lstm, y_pred_rf]).T  # Dự đoán từ LSTM và RF làm input cho meta
y_meta_train = y_rf_test  # Đầu ra từ test của RF

# Huấn luyện mô hình meta (Linear Regression)
meta_model = LinearRegression()
meta_model.fit(X_meta_train, y_meta_train)

# Dự báo từ mô hình meta
X_meta_test = np.vstack([y_pred_lstm, y_pred_rf]).T
y_pred_meta = meta_model.predict(X_meta_test)

# Đánh giá mô hình meta
r2 = r2_score(y_rf_test, y_pred_meta)
rmse = np.sqrt(mean_squared_error(y_rf_test, y_pred_meta))
mae = mean_absolute_error(y_rf_test, y_pred_meta)

print(f"Meta model (Stacking) – R^2: {r2:.9f}, RMSE: {rmse:.3f}, MAE: {mae:.3f}")

# Đánh giá các mô hình cơ bản
print("LSTM – R^2: %.9f, RMSE: %.5f, MAE: %.5f" % (
    r2_score(y_test_seq, y_pred_lstm), 
    np.sqrt(mean_squared_error(y_test_seq, y_pred_lstm)), 
    mean_absolute_error(y_test_seq, y_pred_lstm)
))

print("Random Forest – R^2: %.9f, RMSE: %.3f, MAE: %.3f" % (
    r2_score(y_rf_test, y_pred_rf), 
    np.sqrt(mean_squared_error(y_rf_test, y_pred_rf)), 
    mean_absolute_error(y_rf_test, y_pred_rf)
))


Tổng số mẫu: 5479
Số mẫu train: 4383  | Số mẫu test: 1096


c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Meta model (Stacking) – R^2: 0.998648742, RMSE: 0.564, MAE: 0.298
LSTM – R^2: -0.000758514, RMSE: 15.35708, MAE: 12.62536
Random Forest – R^2: 0.998625966, RMSE: 0.569, MAE: 0.291


# Test RF+LSTM với Best paramester

Best hyperparameters: {'w_lstm': 0.5952718585197654, 'w_rf': 0.4585406693109709}

LSTM - units: 165, learning_rate: 0.0027600131887437498
RF - n_estimators: 300, max_depth: 50, min_samples_split: 2 

In [6]:
import pandas as pd

# Đọc dữ liệu 14 năm từ file Excel
df = pd.read_excel("data_nckh.xlsx", parse_dates=["Time"])
df = df.sort_values("Time")  # sắp xếp theo thời gian

# Kiểm tra và xử lý dữ liệu missing/duplicate
assert df["Time"].is_unique, "Thời gian bị trùng lặp!"  # đảm bảo không trùng
assert df.isnull().sum().sum() == 0, "Có giá trị null trong dữ liệu!"  # đảm bảo không có null

# Lấy cột giá vàng Việt Nam làm biến mục tiêu, các cột khác làm đặc trưng
target_col = "Gold_Price_VN"
feature_cols = ["CPI_VN", "Gold_Price_World", "Oil_Price", "Tỷ giá USD/VND", "VN-Index", 
                "CPI_USA", "SM_M2", "SX_CN", "INTEREST_RATE", "GSI", "GSI_VN"]

# Chuẩn bị dữ liệu cho mô hình
features = df[feature_cols].values
target = df[target_col].values

print("Tổng số mẫu:", len(df))

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, shuffle=False, 
                                                    test_size=0.2)
print("Số mẫu train:", len(X_train), " | Số mẫu test:", len(X_test))

import numpy as np

time_step = 30  # độ dài chuỗi đầu vào cho LSTM (30 ngày)

# Hàm tạo dữ liệu dạng sequence cho LSTM
def create_sequences(X, y, time_step=30):
    X_seq, y_seq = [], []
    for i in range(time_step, len(X)):
        X_seq.append(X[i-time_step:i])    # đoạn [i-time_step ... i-1]
        y_seq.append(y[i])                # giá trị tại thời điểm i (dự báo ở bước kế)
    return np.array(X_seq), np.array(y_seq)

# Tạo dữ liệu cho LSTM (sử dụng tập train)
X_train_seq, y_train_seq = create_sequences(X_train, y_train, time_step)
X_test_seq, y_test_seq = create_sequences(np.vstack([X_train[-time_step:], X_test]), 
                                         np.concatenate([y_train[-time_step:], y_test]), 
                                         time_step)
# Lưu ý: để tạo X_test_seq chính xác, nối đuôi 30 ngày cuối của train với test để không bị thiếu chuỗi
print("Shape X_train_seq:", X_train_seq.shape)  # (n_train_seq, time_step, n_features)

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Xây dựng model LSTM đơn giản
def create_lstm_model(units, lr):
    model = Sequential()
    model.add(LSTM(units=units, activation='tanh', return_sequences=False, input_shape=(time_step, X_train_seq.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse')
    return model

# Huấn luyện mô hình LSTM
lstm_model = create_lstm_model(units=165, lr=0.0027600131887437498)
    
# Dự báo bằng LSTM trên tập test
y_pred_lstm = lstm_model.predict(X_test_seq).flatten()

# Huấn luyện LSTM trên dữ liệu chuỗi
history = lstm_model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=32, 
                         validation_split=0.1, verbose=0)
# Thêm đặc trưng lag-1 của giá vàng (giá vàng ngày trước đó)
X_rf_train = np.hstack([X_train[1:], y_train[:-1, None]])  # ghép cột giá vàng ngày trước vào features ngày hiện tại
y_rf_train = y_train[1:]
X_rf_test = np.hstack([X_test[1:], y_test[:-1, None]])
y_rf_test = y_test[1:]

from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=300, max_depth=50, random_state=0,min_samples_split=2 )
rf_model.fit(X_rf_train, y_rf_train)

# Dự báo bằng LSTM trên tập test (sử dụng X_test_seq đã chuẩn bị)
y_pred_lstm = lstm_model.predict(X_test_seq).flatten()

# Dự báo bằng RF trên tập test
y_pred_rf = rf_model.predict(X_rf_test)

# Kết hợp dự báo (weighted average)
w_lstm, w_rf = 0.5952718585197654, 0.4585406693109709
# Điều chỉnh chiều dài của y_pred_lstm để nó khớp với y_test
y_pred_lstm_adjusted = y_pred_lstm[-len(y_rf_test):]  # Sử dụng độ dài của y_rf_test để cắt y_pred_lstm

# Kết hợp dự báo từ LSTM và RF
y_pred_ensemble = w_lstm * y_pred_lstm_adjusted + w_rf * y_pred_rf  # Đảm bảo độ dài tương ứng với y_test
# Nếu độ dài y_pred_lstm_seq > y_pred_rf (do cách tạo chuỗi), cắt cho khớp với RF (bỏ vài điểm đầu)

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

r2 = r2_score(y_test[-len(y_pred_ensemble):], y_pred_ensemble)
rmse = np.sqrt(mean_squared_error(y_test[-len(y_pred_ensemble):], y_pred_ensemble))
mae = mean_absolute_error(y_test[-len(y_pred_ensemble):], y_pred_ensemble)
print(f"Ensemble model – R^2: {r2:.3f}, RMSE: {rmse:.3f}, MAE: {mae:.3f}")


Tổng số mẫu: 5479
Số mẫu train: 4383  | Số mẫu test: 1096
Shape X_train_seq: (4353, 30, 11)


c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Ensemble model – R^2: 0.678, RMSE: 8.712, MAE: 7.849


In [7]:
import pandas as pd

# Đọc dữ liệu 14 năm từ file Excel
df = pd.read_excel("data_nckh.xlsx", parse_dates=["Time"])
df = df.sort_values("Time")  # sắp xếp theo thời gian

# Kiểm tra và xử lý dữ liệu missing/duplicate
assert df["Time"].is_unique, "Thời gian bị trùng lặp!"  # đảm bảo không trùng
assert df.isnull().sum().sum() == 0, "Có giá trị null trong dữ liệu!"  # đảm bảo không có null

# Lấy cột giá vàng Việt Nam làm biến mục tiêu, các cột khác làm đặc trưng
target_col = "Gold_Price_VN"
feature_cols = ["CPI_VN", "Gold_Price_World", "Oil_Price", "Tỷ giá USD/VND", "VN-Index", 
                "CPI_USA", "SM_M2", "SX_CN", "INTEREST_RATE", "GSI", "GSI_VN"]

# Chuẩn bị dữ liệu cho mô hình
features = df[feature_cols].values
target = df[target_col].values

print("Tổng số mẫu:", len(df))

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, shuffle=False, 
                                                    test_size=0.2)
print("Số mẫu train:", len(X_train), " | Số mẫu test:", len(X_test))

import numpy as np

time_step = 30  # độ dài chuỗi đầu vào cho LSTM (30 ngày)

# Hàm tạo dữ liệu dạng sequence cho LSTM
def create_sequences(X, y, time_step=30):
    X_seq, y_seq = [], []
    for i in range(time_step, len(X)):
        X_seq.append(X[i-time_step:i])    # đoạn [i-time_step ... i-1]
        y_seq.append(y[i])                # giá trị tại thời điểm i (dự báo ở bước kế)
    return np.array(X_seq), np.array(y_seq)

# Tạo dữ liệu cho LSTM (sử dụng tập train)
X_train_seq, y_train_seq = create_sequences(X_train, y_train, time_step)
X_test_seq, y_test_seq = create_sequences(np.vstack([X_train[-time_step:], X_test]), 
                                         np.concatenate([y_train[-time_step:], y_test]), 
                                         time_step)
# Lưu ý: để tạo X_test_seq chính xác, nối đuôi 30 ngày cuối của train với test để không bị thiếu chuỗi
print("Shape X_train_seq:", X_train_seq.shape)  # (n_train_seq, time_step, n_features)

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Xây dựng model LSTM đơn giản
def create_lstm_model(units, lr):
    model = Sequential()
    model.add(LSTM(units=units, activation='tanh', return_sequences=False, input_shape=(time_step, X_train_seq.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse')
    return model

# Huấn luyện mô hình LSTM
lstm_model = create_lstm_model(units=165, lr=0.0027600131887437498)
    
# Dự báo bằng LSTM trên tập test
y_pred_lstm = lstm_model.predict(X_test_seq).flatten()

# Huấn luyện LSTM trên dữ liệu chuỗi
history = lstm_model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=32, 
                         validation_split=0.1, verbose=0)
# Thêm đặc trưng lag-1 của giá vàng (giá vàng ngày trước đó)
X_rf_train = np.hstack([X_train[1:], y_train[:-1, None]])  # ghép cột giá vàng ngày trước vào features ngày hiện tại
y_rf_train = y_train[1:]
X_rf_test = np.hstack([X_test[1:], y_test[:-1, None]])
y_rf_test = y_test[1:]

from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=300, max_depth=50, random_state=0,min_samples_split=2 )
rf_model.fit(X_rf_train, y_rf_train)

# Dự báo bằng LSTM trên tập test (sử dụng X_test_seq đã chuẩn bị)
y_pred_lstm = lstm_model.predict(X_test_seq).flatten()

# Dự báo bằng RF trên tập test
y_pred_rf = rf_model.predict(X_rf_test)

# Kết hợp dự báo (weighted average)
w_lstm, w_rf = 0.5952718585197654, 0.4585406693109709
# Điều chỉnh chiều dài của y_pred_lstm để nó khớp với y_test
y_pred_lstm_adjusted = y_pred_lstm[-len(y_rf_test):]  # Sử dụng độ dài của y_rf_test để cắt y_pred_lstm

# Kết hợp dự báo từ LSTM và RF
y_pred_ensemble = w_lstm * y_pred_lstm_adjusted + w_rf * y_pred_rf  # Đảm bảo độ dài tương ứng với y_test
# Nếu độ dài y_pred_lstm_seq > y_pred_rf (do cách tạo chuỗi), cắt cho khớp với RF (bỏ vài điểm đầu)

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

r2 = r2_score(y_rf_test, y_pred_ensemble)
rmse = np.sqrt(mean_squared_error(y_rf_test, y_pred_ensemble))
mae = mean_absolute_error(y_rf_test, y_pred_ensemble)
print(f"Ensemble model – R^2: {r2:.3f}, RMSE: {rmse:.3f}, MAE: {mae:.3f}")


Tổng số mẫu: 5479
Số mẫu train: 4383  | Số mẫu test: 1096
Shape X_train_seq: (4353, 30, 11)


c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Ensemble model – R^2: 0.678, RMSE: 8.709, MAE: 7.845
